https://gist.github.com/thomasmontoya123/e0237534cbb3d39d092ce67a3f7c273d

In [ ]:
!pip install visualkeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 992.0/992.0 KB 22.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

import visualkeras

In [ ]:
data = keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = data.load_data()

#print the shape of training, testing, and label data
print('Training Data Shape: ', x_train.shape)
print('Testing Data Shape: ', x_test.shape)

print('Label Training Data Shape: ', y_train.shape)
print('Label Testing Data Shape: ', y_test.shape)

170498071/170498071 [==============================] - 3s 0us/step
Training Data Shape:  (50000, 32, 32, 3)
Testing Data Shape:  (10000, 32, 32, 3)
Label Training Data Shape:  (50000, 1)
Label Testing Data Shape:  (10000, 1)


In [ ]:
def preprocess_data(X, Y):
    """
    Pre-processes the data for the model

        :param X: numpy.ndarray of shape (m, 32, 32, 3)
            containing the CIFAR 10 data, where m is the
            number of data points

        :param Y: numpy.ndarray of shape (m,) containing
            the CIFAR 10 labels for X

        :returns: X_p, Y_p
    """
    X_p = tf.keras.applications.densenet.preprocess_input(X)

    # encode to one-hot
    Y_p = tf.keras.utils.to_categorical(Y, 10)
    return X_p, Y_p

In [ ]:
CALLBACKS = []
MODEL_PATH = 'cifar10.h5'
optimizer = tf.keras.optimizers.Adam()


# pre-procces data
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [ ]:
# input tensor
inputs = tf.keras.Input(shape=(32, 32, 3))

# upscale layer
upscale = tf.keras.layers.Lambda(lambda x: tf.image.resize_with_pad(x,
                                                                  160,
                                                                  160,
                                                                  method=tf.image.ResizeMethod.BILINEAR))(inputs)

In [ ]:
# load base model
base_model = tf.keras.applications.DenseNet121(include_top=False,
                                        weights='imagenet',
                                        input_tensor=upscale,
                                        input_shape=(160,160,3),
                                        pooling='max')
# add top layers
out = base_model.output
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(64, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = tf.keras.layers.Dense(10, activation='softmax')(out)

29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
# callbacks
CALLBACKS.append(tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_PATH,
                                              monitor='val_accuracy',
                                              save_best_only=True))

CALLBACKS.append(tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                            verbose=1,
                                            patience=5))

CALLBACKS.append(tf.keras.callbacks.TensorBoard(log_dir='logs'))

In [ ]:
# model compile
model = tf.keras.models.Model(inputs=inputs, outputs=out)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# train
model.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

Epoch 1/5
391/391 [==============================] - 379s 765ms/step - loss: 0.7974 - accuracy: 0.7467 - val_loss: 0.5286 - val_accuracy: 0.8426
Epoch 2/5
391/391 [==============================] - 291s 743ms/step - loss: 0.3750 - accuracy: 0.8868 - val_loss: 0.3817 - val_accuracy: 0.8776
Epoch 3/5
391/391 [==============================] - 290s 743ms/step - loss: 0.2892 - accuracy: 0.9109 - val_loss: 0.3928 - val_accuracy: 0.8808
Epoch 4/5
391/391 [==============================] - 291s 743ms/step - loss: 0.2330 - accuracy: 0.9276 - val_loss: 0.3796 - val_accuracy: 0.8750
Epoch 5/5
391/391 [==============================] - 284s 726ms/step - loss: 0.1960 - accuracy: 0.9387 - val_loss: 0.4102 - val_accuracy: 0.8842


In [ ]:
# model compile
model1 = tf.keras.models.Model(inputs=inputs, outputs=out)

model1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# train
model1.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=100,
          callbacks=CALLBACKS,
          validation_data=(x_test, y_test))

Epoch 1/100
391/391 [==============================] - 304s 735ms/step - loss: 0.1725 - accuracy: 0.9473 - val_loss: 0.3323 - val_accuracy: 0.8984
Epoch 2/100
391/391 [==============================] - 290s 741ms/step - loss: 0.1592 - accuracy: 0.9507 - val_loss: 0.3885 - val_accuracy: 0.8909
Epoch 3/100
391/391 [==============================] - 291s 745ms/step - loss: 0.1389 - accuracy: 0.9569 - val_loss: 0.3176 - val_accuracy: 0.9057
Epoch 4/100
391/391 [==============================] - 284s 727ms/step - loss: 0.1315 - accuracy: 0.9595 - val_loss: 0.3128 - val_accuracy: 0.9066
Epoch 5/100
391/391 [==============================] - 290s 741ms/step - loss: 0.1090 - accuracy: 0.9672 - val_loss: 0.3513 - val_accuracy: 0.9049
Epoch 6/100
391/391 [==============================] - 284s 728ms/step - loss: 0.1076 - accuracy: 0.9663 - val_loss: 0.2497 - val_accuracy: 0.9250
Epoch 7/100
391/391 [==============================] - 290s 741ms/step - loss: 0.0937 - accuracy: 0.9708 - val_loss: 0

In [ ]:
model1.evaluate(x_test, y_test, batch_size=128, verbose=1)

79/79 [==============================] - 14s 172ms/step - loss: 0.3477 - accuracy: 0.9108


[0.34768593311309814, 0.9107999801635742]

In [ ]:
# model compile
model2 = tf.keras.models.Model(inputs=inputs, outputs=out)

model2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# train
model2.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=150,
          callbacks=CALLBACKS,
          validation_data=(x_test, y_test))

Epoch 1/150
391/391 [==============================] - 300s 728ms/step - loss: 0.0541 - accuracy: 0.9828 - val_loss: 0.3269 - val_accuracy: 0.9163
Epoch 2/150
391/391 [==============================] - 289s 739ms/step - loss: 0.0468 - accuracy: 0.9853 - val_loss: 0.3563 - val_accuracy: 0.9146
Epoch 3/150
391/391 [==============================] - 290s 742ms/step - loss: 0.0476 - accuracy: 0.9852 - val_loss: 0.2866 - val_accuracy: 0.9259
Epoch 4/150
391/391 [==============================] - 289s 739ms/step - loss: 0.0500 - accuracy: 0.9847 - val_loss: 0.3901 - val_accuracy: 0.9046
Epoch 5/150
391/391 [==============================] - 290s 742ms/step - loss: 0.0404 - accuracy: 0.9876 - val_loss: 0.2696 - val_accuracy: 0.9327
Epoch 6/150
391/391 [==============================] - 289s 738ms/step - loss: 0.0450 - accuracy: 0.9869 - val_loss: 0.2657 - val_accuracy: 0.9307
Epoch 7/150
391/391 [==============================] - 288s 738ms/step - loss: 0.0375 - accuracy: 0.9890 - val_loss: 0

In [ ]:
# model compile
model3 = tf.keras.models.Model(inputs=inputs, outputs=out)

model3.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# train
model3.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=200,
          callbacks=CALLBACKS,
          validation_data=(x_test, y_test))

Epoch 1/200
391/391 [==============================] - 299s 726ms/step - loss: 0.0314 - accuracy: 0.9908 - val_loss: 0.4682 - val_accuracy: 0.8996
Epoch 2/200
391/391 [==============================] - 282s 721ms/step - loss: 0.0386 - accuracy: 0.9887 - val_loss: 0.3127 - val_accuracy: 0.9312
Epoch 3/200
391/391 [==============================] - 288s 738ms/step - loss: 0.0320 - accuracy: 0.9903 - val_loss: 0.2833 - val_accuracy: 0.9311
Epoch 4/200
391/391 [==============================] - 282s 720ms/step - loss: 0.0330 - accuracy: 0.9901 - val_loss: 0.3462 - val_accuracy: 0.9182
Epoch 5/200
391/391 [==============================] - 289s 739ms/step - loss: 0.0316 - accuracy: 0.9901 - val_loss: 0.3249 - val_accuracy: 0.9224
Epoch 6/200
391/391 [==============================] - 282s 720ms/step - loss: 0.0311 - accuracy: 0.9904 - val_loss: 0.3014 - val_accuracy: 0.9280
Epoch 7/200
391/391 [==============================] - 288s 738ms/step - loss: 0.0307 - accuracy: 0.9908 - val_loss: 0

In [ ]:
model1.evaluate(x_test, y_test, batch_size=128, verbose=1)

79/79 [==============================] - 14s 172ms/step - loss: 0.2953 - accuracy: 0.9287


[0.29532909393310547, 0.9286999702453613]

In [ ]:
model1.evaluate(x_test, y_test, batch_size=128, verbose=1)

79/79 [==============================] - 14s 178ms/step - loss: 0.2953 - accuracy: 0.9287


[0.29532909393310547, 0.9286999702453613]

In [ ]:
from sklearn.metrics import confusion_matrix

# make predictions on the test set
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

# convert y_test to 1D array
y_test = np.argmax(y_test, axis=1)

# calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n", cm)

313/313 [==============================] - 17s 48ms/step
Confusion matrix: 
 [[907   3  27   4  11   1  12   3  31   1]
 [ 11 943   0   2   1   0   3   1  11  28]
 [  5   1 923   9  10  15  33   3   1   0]
 [  2   0  20 820  22  78  46   7   5   0]
 [  0   1  22   6 947   5  13   4   2   0]
 [  1   0  15  45  16 898  10  14   1   0]
 [  0   0   4   2   2   3 988   1   0   0]
 [  3   0   6   5  18  11   1 954   2   0]
 [ 20   1  10   1   0   1   2   0 960   5]
 [ 14  16   4   4   1   1   7   1  18 934]]
